In [1]:
import transformers
import pandas
import numpy
import torch
import json

/home/jerko/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = pandas.read_parquet('raw_data/RACE/train-00000-of-00001.parquet')

In [3]:
dataset

,example_id,article,answer,question,options
0,high19088.txt,Last week I talked with some of my students ab...,C,We can know from the passage that the author w...,"[doctor, model, teacher, reporter]"
1,high19088.txt,Last week I talked with some of my students ab...,C,Many graduates today turn to cosmetic surgery ...,"[marry a better man/woman, become a model, get..."
2,high19088.txt,Last week I talked with some of my students ab...,D,"According to the passage, the author believes ...","[everyone should purchase perfection, whatever..."
3,high19088.txt,Last week I talked with some of my students ab...,B,Which' s the best title for the passage?.,"[Young Graduates Have Higher Expectations, You..."
4,high15596.txt,"YUZHOU, HENAN -An accident in a central China ...",B,What could be the best title for this passage?,"[Death Toll Rises in an Accident in China, A C..."
...,...,...,...,...,...
87861,middle2313.txt,"Last year when Tom graduated from school, he c...",B,Tom came to the station to _ .,"[find a job, take the train home, sell the tic..."
87862,middle2313.txt,"Last year when Tom graduated from school, he c...",C,The woman walked to Tom because _ .,"[she was Tom's old friend, her child wanted to..."
87863,middle2313.txt,"Last year when Tom graduated from school, he c...",D,Tom didn't take the train because _ .,"[he didn't take a train ticket, he missed the ..."
87864,middle2313.txt,"Last year when Tom graduated from school, he c...",B,The old man wanted to _ .,"[lend Tom some money, give Tom a job, say ""Tha..."


In [4]:
with open('raw_data/ReCoRD/train.json') as f:
    record_dataset = json.load(f)

In [5]:
record_dataset = pandas.DataFrame(record_dataset['data'])

In [6]:
record_dataset

,id,source,passage,qas
0,536e2fdfb3c986efa15da314b0df2d2d218cf693,Daily mail,{'text': 'The harrowing stories of women and c...,[{'id': '536e2fdfb3c986efa15da314b0df2d2d218cf...
1,f15689cd256daa03fcfd8c357f1376a8a7017b64,CNN,"{'text': 'Caracas, Venezuela (CNN) -- It's bee...",[{'id': 'f15689cd256daa03fcfd8c357f1376a8a7017...
2,bd4f4fd84a92411a7c03fe80fa2e84270ac51ac3,Daily mail,{'text': 'U.S. Secretary of State Henry Kissin...,[{'id': 'bd4f4fd84a92411a7c03fe80fa2e84270ac51...
3,16f4d0c2b42cfd57557d1aaf398110ebf519874f,Daily mail,{'text': 'A Peruvian tribe once revered by the...,[{'id': '16f4d0c2b42cfd57557d1aaf398110ebf5198...
4,a204b0da2b9a5267a0527632ab8d5691b2c412de,Daily mail,{'text': 'For everyone who has ever thought ab...,[{'id': 'a204b0da2b9a5267a0527632ab8d5691b2c41...
...,...,...,...,...
65704,6079a996b955d3130ad59c37b57d9958e0539b83,Daily mail,{'text': 'Liverpool are ready to revive their ...,[{'id': '6079a996b955d3130ad59c37b57d9958e0539...
65705,4e466fabc27adf6cb2180c8441f9b0d5266062ba,Daily mail,{'text': 'A Russian tourist has been hospitali...,[{'id': '4e466fabc27adf6cb2180c8441f9b0d526606...
65706,cc54a42ae756fa5bddda37e91948149793bd072b,CNN,{'text': 'Editor's note: Ilya Shapiro is a sen...,[{'id': 'cc54a42ae756fa5bddda37e91948149793bd0...
65707,5c6f7eec2a86ccd30d412c24f90f4c1e91aa607b,CNN,{'text': '(CNN) -- Newt Gingrich quit the pres...,[{'id': '5c6f7eec2a86ccd30d412c24f90f4c1e91aa6...


In [7]:
filtered_rows = record_dataset[record_dataset['id'] == '6079a996b955d3130ad59c37b57d9958e0539b83']
filtered_rows

,id,source,passage,qas
65704,6079a996b955d3130ad59c37b57d9958e0539b83,Daily mail,{'text': 'Liverpool are ready to revive their ...,[{'id': '6079a996b955d3130ad59c37b57d9958e0539...


In [8]:
record_dataset['qas'][74]

[{'id': '33ae938f21c5e4118caa4109967e177e540f1c37-2db27135766475e5e4554128801bb5fab53e80ce-72',
  'query': "But here's what the three main players -- the United States, Russia and @placeholder -- are really thinking.",
  'answers': [{'start': 342, 'end': 347, 'text': 'Turkey'},
   {'start': 651, 'end': 655, 'text': 'Turks'},
   {'start': 701, 'end': 707, 'text': 'Turkish'}]}]

In [9]:
with open('raw_data/multirc-v2/splitv2/train_456-fixedIds.json') as f:
    rc_dataset = json.load(f)

In [10]:
rc_dataset = pandas.DataFrame(rc_dataset['data'])

In [11]:
rc_dataset

,paragraph,id
0,{'text': '<b>Sent 1: </b>Animated history of t...,Society_Law_and_Justice/masc-A_defense_of_Mich...
1,{'text': '<b>Sent 1: </b>Before the mysterious...,News/CNN/cnn-3b372dbbdea7cabac969352b8b3169a0e...
2,{'text': '<b>Sent 1: </b>Hotel California My f...,Fiction-stories-masc-hotel-California-1.txt
3,{'text': '<b>Sent 1: </b>Elaan is a declaratio...,wikiMovieSummaries-27380172.txt
4,{'text': '<b>Sent 1: </b>Not until 1998 did al...,Sept11-reports/oanc-chapter-2-5.txt
...,...,...
451,{'text': '<b>Sent 1: </b>How would the univers...,Science-textbook/science-g3-31.txt
452,{'text': '<b>Sent 1: </b>Once upon a time ther...,Fiction/mctest-mc160.dev.5-0.txt
453,{'text': '<b>Sent 1: </b>In her storage room-t...,Society_Law_and_Justice/oanc-Crains_New_York_B...
454,{'text': '<b>Sent 1: </b>A West Tennessee nonp...,Society_Law_and_Justice/oanc-fight_domestic_ab...


In [12]:
rc_dataset['paragraph'][0].keys()

dict_keys(['text', 'questions'])

In [13]:
import re

text = rc_dataset['paragraph'][0]['text']

pattern = r"<b>Sent \d+: </b>(.*?)<br>"
sentences = re.findall(pattern, text)

# Print the sentences
for i, sentence in enumerate(sentences, 1):
    print(f"Sentence {i}: {sentence}")


Sentence 1: Animated history of the US.
Sentence 2: Of course the cartoon is highly oversimplified, and most critics consider it one of the weakest parts of the film.
Sentence 3: But it makes a valid claim which you ignore entirely: That the strategy to promote "gun rights" for white people and to outlaw gun possession by black people was a way to uphold racism without letting an openly terrorist organization like the KKK flourish.
Sentence 4: Did the 19th century NRA in the southern states promote gun rights for black people?
Sentence 5: I highly doubt it.
Sentence 6: But if they didn't, one of their functions was to continue the racism of the KKK.
Sentence 7: This is the key message of this part of the animation, which is again being ignored by its critics.
Sentence 8: Buell shooting in Flint.
Sentence 9: You write: "Fact: The little boy was the class thug, already suspended from school for stabbing another kid with a pencil, and had fought with Kayla the day before".
Sentence 10: Th

In [14]:
rc_dataset['paragraph'][6]['questions'][5]

{'question': 'After what successfully won battle was Philip named "Hegemon"',
 'sentences_used': [9, 10, 1, 8],
 'answers': [{'text': 'Modern scholars', 'isAnswer': False, 'scores': {}},
  {'text': 'The battle of Sparta', 'isAnswer': False, 'scores': {}},
  {'text': 'Battle of Chaeronea', 'isAnswer': True, 'scores': {}}],
 'idx': '5',
 'multisent': True}

In [15]:
with open('raw_data/BoolQ/train.jsonl') as f:
    boolq_dataset = f.readlines()

boolq_dataset = pandas.DataFrame([json.loads(x) for x in boolq_dataset])

In [16]:
boolq_dataset

,question,title,answer,passage
0,do iran and afghanistan speak the same language,Persian language,True,"Persian (/ˈpɜːrʒən, -ʃən/), also known by its ..."
1,do good samaritan laws protect those who help ...,Good Samaritan law,True,Good Samaritan laws offer legal protection to ...
2,is windows movie maker part of windows essentials,Windows Movie Maker,True,Windows Movie Maker (formerly known as Windows...
3,is confectionary sugar the same as powdered sugar,Powdered sugar,True,"Powdered sugar, also called confectioners' sug..."
4,is elder scrolls online the same as skyrim,The Elder Scrolls Online,False,As with other games in The Elder Scrolls serie...
...,...,...,...,...
9422,is a us district court a federal court,United States district court,True,The United States district courts are the gene...
9423,can a tenant get a restraining order against a...,Landlord harassment,True,"If a landlord is found to be retaliating, he o..."
9424,is the golden state warriors in the playoffs,Golden State Warriors,True,The Warriors went into the 2018 playoffs as th...
9425,downton abbey will there be a season 7,List of Downton Abbey episodes,False,Downton Abbey is a British period drama televi...


In [17]:
boolq_dataset['answer'][42] == False

True

In [18]:
from datasets import Boolq_dataset
from dataset_loaders import load_boolq
from transformers import BertTokenizer

boolq_dataset = load_boolq("raw_data/BoolQ/train.jsonl", BertTokenizer.from_pretrained('bert-base-uncased'))


Token indices sequence length is longer than the specified maximum sequence length for this model (910 > 512). Running this sequence through the model will result in indexing errors


In [19]:
boolq_dataset[435]

(tensor([[  101,  1996, 28833, 15790, 13170,  1006, 26632, 15721,  7274,  8915,
           7382,  2378, 18009,  2072,  1007,  2003,  1037,  2427,  1997, 13170,
           1999,  1996,  2155, 18178,  2135, 13626,  6096,  1010,  3128,  2000,
          12573, 10746,  1999,  1996,  2142,  2163,  1012,  1049,  1012,  8915,
           7382,  2378, 18009,  2072,  2003,  2028,  1997,  1996, 26858, 12573,
          16489,  1999,  1996,  2088,  1012,  2009,  2003,  2411,  3378,  2007,
           1010,  2021,  2025,  4876,  3141,  2000,  1010,  1996,  2691, 15790,
          13170,  1010,  2029,  2003,  1999,  1996,  3562, 18178,  2135,  7265,
           1012,  1996,  3563, 19626,  8915,  7382,  2378, 18009,  2072,  2003,
           1999,  3932,  1997,  3803,  9201, 10727, 24873, 16118, 11057,  2094,
           6213,  8915,  7382,  2378,  3600,  1012,  2003,  2045,  2107,  2518,
           2004,  2019, 28833, 15790, 13170,  1029,  2748,  1012,   102]]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [20]:
from datasets import MultiRC_dataset
from dataset_loaders import load_multirc
from transformers import BertTokenizer

multirc_dataset = load_multirc("raw_data/multirc-v2/splitv2/train_456-fixedIds.json", BertTokenizer.from_pretrained('bert-base-uncased'))

Token indices sequence length is longer than the specified maximum sequence length for this model (513 > 512). Running this sequence through the model will result in indexing errors


In [21]:
multirc_dataset[56]

(tensor([[  101,  4918, 21442, 22231,  2638,  2018,  2053,  7669,  1999, 12329,
           2010,  2713,  2043,  6262,  2234,  2006,  4611,  2012,  2416,  1051,
           1005,  5119,  1012,  2002,  2170,  2039,  1037,  9298,  1998,  2253,
           2012,  2320,  2000,  2010,  4734,  2012,  1996,  7987,  5602, 23567,
           2063,  1025,  1998,  9666, 15010,  2628,  2032,  1012,  2096,  2002,
          15105,  2010,  4377,  1011,  5435,  1010,  2165,  1037,  7198,  1998,
           2123,  7228,  2010,  3788,  4848, 21442, 22231,  2638,  2001,  1999,
           1037,  2829,  2817,  1012,  2847,  3283,  8227,  2018,  2042,  9689,
           5565,  2012,  1996,  2264,  4589,  2160,  1998,  2872,  1999,  1996,
           2729,  1997,  2214, 10602,  8292, 29346,  1010,  2040,  2052,  3457,
           2014,  2066,  2019, 13958,  2890,  1012,  2045,  2001,  2053,  6234,
           2342,  1997,  2010, 24748,  5582,  2044,  2014,  1010,  1998,  2010,
           6545,  1998,  1996,  5456,  1

In [22]:
multirc_dataset.question_answer_objs[0].question

'Does the author claim the animated films message is that the NRA upholds racism?'

In [23]:
multirc_dataset.passages[0]

['Animated history of the US.',
 'Of course the cartoon is highly oversimplified, and most critics consider it one of the weakest parts of the film.',
 'But it makes a valid claim which you ignore entirely: That the strategy to promote "gun rights" for white people and to outlaw gun possession by black people was a way to uphold racism without letting an openly terrorist organization like the KKK flourish.',
 'Did the 19th century NRA in the southern states promote gun rights for black people?',
 'I highly doubt it.',
 "But if they didn't, one of their functions was to continue the racism of the KKK.",
 'This is the key message of this part of the animation, which is again being ignored by its critics.',
 'Buell shooting in Flint.',
 'You write: "Fact: The little boy was the class thug, already suspended from school for stabbing another kid with a pencil, and had fought with Kayla the day before".',
 "This characterization of a six-year-old as a pencil-stabbing thug is exactly the kind

In [25]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer.decode([  1029, 2009, 8440, 1005,
         1056, 6003, 2007, 2200, 2204, 7022,  102])

"? it hasn't emerged with very good grades [SEP]"